In [1]:
# import stats packages
import numpy as np
import pandas as pd
import sys, os
import glob
import fiona
from plotnine import ggplot

# import spatial modules
import ee
import geemap
#from osgeo import gdal #install with pip3, don't use conda

# don't display warnings
import warnings
warnings.filterwarnings('ignore') 

In [2]:
#********************************* Initialize GEE  ***********************************#
## trigger the authentication flow. only need once
ee.Authenticate()
## After inserting the API key initialize GEE
ee.Initialize()

Enter verification code:  4/1AX4XfWgEBOd2Wop2nehzuIK4vrS44YWHKLFXUSyJP2QQd6FanS2B1sNHcms



Successfully saved authorization token.


In [53]:
#****************************** define user parameters *******************************#
# setup output directory
outDIR="/Users/darylyang/Desktop/conda/gee/test"
# create output folder if it does not exist
os.makedirs(outDIR, exist_ok=True)
# print output directory
print(outDIR)

# setup date range in which MODIS data will be extracted
dateBEG = ee.Date("2011-01-01")
dateEND = ee.Date("2015-12-31")
# setup month in which MODIS data will be extracted 
monBEG = 1
monEND = 1

# set visualization parameters.
lst_vis_params = {
    'min': 1,
    'max': 255,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
}

/Users/darylyang/Desktop/conda/gee/test


In [54]:
#********************************* load in data **************************************#
# load in modis land surface temperature
modCOLL = ee.ImageCollection('MODIS/006/MOD11A1').filter(ee.Filter.date(dateBEG, dateEND))
modCOLL = modCOLL.filter(ee.Filter.calendarRange(monBEG,monEND,'month'));
# count the number of images found
count = modCOLL.size()
print('Count: ', str(count.getInfo())+'\n')
# extract temperature
modEMIS_31 = modCOLL.select('Emis_31')
modEMIS_32 = modCOLL.select('Emis_32')

# load in a polygon where MODIS will be clipped out
shpDIR = "/Volumes/data2/dyang/projects/arctic_energy/boundary/above_domain_use.shp"
shpVCT = geemap.shp_to_ee(shpDIR)

Count:  155



In [55]:
#******************************** preprocessing *************************************#
# clip out the region defined by shp file
def geeclip (image):
    return image.clip(shpVCT)
modEMIS_31_CLP = modEMIS_31.map(geeclip)
modEMIS_32_CLP = modEMIS_32.map(geeclip)

# calculate mean
# calculate mean
modEMIS_31_MN = modEMIS_31_CLP.reduce(ee.Reducer.mean())
modEMIS_32_MN = modEMIS_32_CLP.reduce(ee.Reducer.mean())

modEMIS_MN = modEMIS_31_MN.add(modEMIS_32_MN).divide(2)

# make a quick display show the data region
Map = geemap.Map(center=(65, -120), zoom=3)
Map.addLayer(shpVCT, {}, 'ROI')
Map.addLayer(modEMIS_MN, lst_vis_params, 'MODIS LST', True, 1)
Map.add_colorbar(vis_params=lst_vis_params, label='MODIS LST (Kelvin, Scale = 0.02)')
Map

Map(center=[65, -120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [56]:
#******************************** preprocessing *************************************#
geometry = ee.Geometry.Rectangle([-168, 49, -81, 80])
#geometry = shpVCT.geometry().bounds().getInfo()['coordinates']
task = ee.batch.Export.image.toDrive(**{
    'image': modEMIS_MN, 
    'description': 'modis_emis_above_month_1', 
    'folder': 'modis_emis', 
    'scale': 1000, 
    'region': geometry, #shpVCT.geometry().bounds().getInfo()['coordinates'], 
    'maxPixels': 10e12 })
task.start()